# Instalando dependências

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    !git clone https://github.com/LucaLemos/UFRPE_AprendizagemReforco
    sys.path.append("/content/UFRPE_AprendizagemReforco")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

In [ ]:
if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg
    !pip install gymnasium==1.0.0   # conferir se precisa
    #!pip install tianshou # Para criar o Replay_Buffer
    #!pip install d3rlpy==2.7.0
    # clone repository

# Criando Dataset

In [ ]:
import gymnasium as gym
import torch
from util.algorithms import run_sarsa
from util.network import ReplayBuffer, MLP, epsilon_greedy_qnet, calc_loss
from IPython.display import clear_output

In [ ]:
DATASET_SIZE = 200_000  # Tamanho do conjunto de dados (replay buffer)
LEARNING_RATE = 1e-3  # Taxa de aprendizado para o otimizador
GAMMA = 0.99  # Fator de desconto
BATCH_SIZE = 128  # Tamanho do batch para treinamento da rede neural

In [ ]:
ENV_NAMES = []
ENVS_REPLAY_BUFFER = []

## Dataset Básico

In [ ]:
# Passo 1: Coletar um conjunto fixo de transições (Replay Buffer)
# Ambientes Básicos
ENV_NAMES += ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0"]
for i, env_name in enumerate(ENV_NAMES):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    env = gym.make(env_name, render_mode="rgb_array")
    
    sum_rewards_per_ep, q = run_sarsa(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer, sum_rewards_per_ep, q))
    replay_buffer.save_config(f"config\dataset\sarsa\{env_name}.json")


## Dataset com Rede Neural

In [ ]:
from torch import optim, nn
import numpy as np

In [ ]:
def DQN_TRAIN(env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_buffer, batch_size, epsilon_f, epsilon_decay_period, transitions):
    # Cria o otimizador, que vai fazer o ajuste dos pesos da 'qnet',
    # Usa uma técnica de gradiente descendente de destaque, chamada ADAM
    optimizer = optim.Adam(qnet.parameters(), lr=qnet_lr)

    episode_reward_list = []
    step = 0
    epsilon = 1.0

    state, _ = env.reset()
    episode_reward = 0.0

    for _ in range(transitions):
        # Decaimento linear do epsilon
        epsilon = max(epsilon_f, 1.0 - step / epsilon_decay_period)

        action = epsilon_greedy_qnet(qnet, env, state, epsilon)

        # Faz um passo / Aplica uma ação no ambiente
        new_state, reward, terminated, truncated, _ = env.step(action)

        step += 1
        done = terminated or truncated
        episode_reward += reward

        # Adiciona no buffer
        replay_buffer.add(state, action, reward, terminated, new_state)
        state = new_state

        if done:
            episode_reward_list.append(episode_reward)
            if len(episode_reward_list) % 100 == 0:
                mean_reward = np.mean(episode_reward_list[-100:])
                print(f'Episode Reward: {episode_reward_list[-1]} | Epsilon: {epsilon} | Step: {step} | Mean Reward: {mean_reward}')
            state, _ = env.reset()
            episode_reward = 0.0

            # Abaixo, faz vários loggings de dados
            
        # Faz a 'tgt_net' receber os mesmos valores de pesos da 'qnet', na frequência indicada
        
        if step > batch_size:
            if step % target_update_freq == 0:
                target_qnet.load_state_dict(qnet.state_dict())
            # Escolhe amostras aleatórios do buffer e faz uma atualização dos pesos da rede
            optimizer.zero_grad()
            batch = replay_buffer.sample_continuous()
            loss_t = calc_loss(batch, qnet, target_qnet, gamma)
            loss_t.backward()
            optimizer.step()
            
    mean_reward = np.mean(episode_reward_list[-100:])
    print(f"Mean_reward: {mean_reward}")
    filename = f"./neural_dataset/{env_name}.pth"
    torch.save(qnet.state_dict(), filename)
    print(f"Model saved as {filename}")

In [ ]:
GOAL_REWARD = 200
GAMMA = 0.999
REPLAY_SIZE = 100_000
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
SYNC_TARGET_FRAMES = 1_000

EPSILON_DECAY_PERIOD = 75_000
EPSILON_FINAL = 2e-2

In [ ]:
#ENV_NAMES += ["CartPole-v1"]
ENV_NAMES += ["LunarLander-v3"]
for i, env_name in enumerate(ENV_NAMES[-1:]):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    print(env_name)
    env = gym.make(env_name)
    
    qnet1 = MLP(env.observation_space.shape[0], env.action_space.n, [256,256])
    qtarget1 = MLP(env.observation_space.shape[0], env.action_space.n, [256,256])

    DQN_TRAIN(
    env = env,
    env_name = env_name,
    gamma = GAMMA,
    qnet = qnet1,
    qnet_lr = LEARNING_RATE,
    target_qnet = qtarget1,
    target_update_freq = SYNC_TARGET_FRAMES,
    replay_buffer= buffer,
    batch_size = BATCH_SIZE,
    epsilon_f = EPSILON_FINAL,
    epsilon_decay_period = EPSILON_DECAY_PERIOD,
    transitions = REPLAY_SIZE)

    #!sum_rewards_per_ep, q = run_sarsa(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, buffer))
    buffer.save_config(f"config\dataset\dqn\{env_name}.json")

# Treinando o Modelo

## Carregando Datasets

In [1]:
import gymnasium as gym
from util.network import ReplayBuffer


In [2]:
ENV_NAMES = ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0", "CartPole-v1", "LunarLander-v3"]

In [3]:
ENVS_REPLAY_BUFFER = []
for env_name in ENV_NAMES[:3]:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\sarsa\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))
for env_name in ENV_NAMES[3:]:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\dqn\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))

## Definições de Treino

In [4]:
import argparse

In [5]:
def get_config(env_name, count_episodes, seed, gamma, tau, alpha, alpha_decay, lr, steps, hidden_size, isDiscrete):
    parser = argparse.ArgumentParser(description='RL')
    parser.add_argument("--run_name", type=str, default=f"{env_name}-DQN-CQL", help="Run name, default: CQL-DQN")
    parser.add_argument("--env", type=str, default=env_name, help="Gym environment name, default: CartPole-v0")
    parser.add_argument("--episodes", type=int, default=count_episodes, help="Number of episodes, default: 200")
    parser.add_argument("--seed", type=int, default=seed, help="Seed, default: 1")
    parser.add_argument("--steps", type=int, default=steps, help="Saves the network every x epochs, default: 25")
    
    parser.add_argument("--gamma", type=float, default=gamma, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--tau", type=float, default=tau, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--alpha", type=float, default=alpha, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--alpha_decay", type=float, default=alpha_decay, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--lr", type=float, default=lr, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--hidden_size", type=list[int], default=hidden_size, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--isDiscrete", type=bool, default=isDiscrete, help="Saves the network every x epochs, default: 25")
    
    args, _ = parser.parse_known_args()
    return args

In [6]:
from util.network import CQLAgent, to_one_hot, MLP
import numpy as np
import random
import torch
from collections import deque

In [7]:
def train_DQN_CQL(config, buffer):
    np.random.seed(config.seed)
    random.seed(config.seed)
    torch.manual_seed(config.seed)
    env = gym.make(config.env)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    losses = deque(maxlen=1000)
    if config.isDiscrete:
        agent = CQLAgent(env.observation_space.n, env.action_space.n, config.tau, config.gamma, config.lr, config.alpha, config.alpha_decay, config.hidden_size, device, config.isDiscrete)
    else:
        agent = CQLAgent(env.observation_space.shape[0], env.action_space.n, config.tau, config.gamma, config.lr, config.alpha, config.alpha_decay, config.hidden_size, device, config.isDiscrete)
    for i in range(config.steps):
        loss, cql_loss, bellmann_error = agent.learn(buffer.sample_continuous())
        losses.append(loss)
        agent.alpha = max(0.3, agent.alpha - agent.alpha_decay)         
        if i % 1000 == 0:
            print(f"[TRAIN {i}] Mean Q Loss: {np.mean(losses)} ")
    
    agent.save(config, save_name="CQL-DQN")
    
    return env

In [8]:
def evaluate(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0
        while not done:
            state_tensor = torch.tensor([state], dtype=torch.long)
            state_tensor = to_one_hot(model.state_size, state_tensor)[0].float().unsqueeze(0)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()
            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [9]:
# Função para extrair a tabela Q da rede neural
def extract_q_table(q_network, num_states, num_actions):
    q_table = np.zeros((num_states, num_actions))
    for state in range(num_states):
        state_tensor = torch.tensor([state], dtype=torch.long)
        state_tensor = to_one_hot(num_states, state_tensor)[0].float().unsqueeze(0)
        q_values = q_network(state_tensor).detach().numpy()
        q_table[state] = q_values
    return q_table

In [10]:
from util.qtable_helper import record_video_qtable
from util.notebook import display_videos_from_path

In [11]:

def extract_and_display(model, env_name):
    # Extrair a tabela Q da rede neural treinada
    q_table = extract_q_table(model, model.state_size, model.action_size)

    # Gravar um vídeo da política treinada
    video_path = 'videos/'  # Pasta onde os vídeos serão salvos
    video_prefix = f"cql_{env_name}"  # Prefixo para o nome do arquivo de vídeo

    # Gravar o vídeo
    record_video_qtable(env_name, q_table, episodes=2, folder=video_path, prefix=video_prefix)

    # Exibir o vídeo gravado
    display_videos_from_path(video_path, prefix=video_prefix)

In [12]:
def evaluate_Continuous(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0.0
        step = 0
        while not done:
            state_v = torch.tensor(state, dtype=torch.float32)
            state_v = state_v.unsqueeze(0)  # Adiciona dimensão de batch como eixo 0 (e.g. transforma uma lista [a,b,c] em [[a,b,c]])
            with torch.no_grad():
                q_vals_v = model(state_v)
                _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())


            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [13]:
# Função para renderizar o ambiente com o modelo treinado
import time
def render_environment(model, env_name, device="cpu", episodes=5, max_steps=100000):
    env = gym.make(env_name, render_mode="human")  # Modo "human" para renderização
    for episode in range(episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0

        while not done and step < max_steps:
            # Renderiza o ambiente
            env.render()
            time.sleep(0.02)  # Adiciona um pequeno delay para visualização

            # Converte o estado para um tensor e passa pelo modelo
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()

            # Executa a ação no ambiente
            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1

            if done or truncated:
                print(f"Episódio {episode + 1}: Recompensa = {episode_reward}, Passos = {step}")
                break

    env.close()

## FrozenLake

In [14]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 32
STEPS = 100_000
GAMMA = 0.95 # Fator de desconto
TAU = 1e-1      # Taxa de atualização da target_network
ALPHA = 0.9      # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 3e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [64, 32]
ISDISCRETE = True
# Extra:

In [15]:
ENVS_REPLAY_BUFFER[0][2].batch_size = BATCH_SIZE

In [16]:
config = get_config(ENVS_REPLAY_BUFFER[0][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [17]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[0][2])

[TRAIN 0] Mean Q Loss: 1.4393583536148071 
[TRAIN 1000] Mean Q Loss: 0.7371770503222942 
[TRAIN 2000] Mean Q Loss: 0.6102223298847675 
[TRAIN 3000] Mean Q Loss: 0.594596025750041 
[TRAIN 4000] Mean Q Loss: 0.5902134938538075 
[TRAIN 5000] Mean Q Loss: 0.5990718638002872 
[TRAIN 6000] Mean Q Loss: 0.5923053021878004 
[TRAIN 7000] Mean Q Loss: 0.5864106568396091 
[TRAIN 8000] Mean Q Loss: 0.5993010680526495 
[TRAIN 9000] Mean Q Loss: 0.5763265080153942 
[TRAIN 10000] Mean Q Loss: 0.5841210248321295 
[TRAIN 11000] Mean Q Loss: 0.5845037899762392 
[TRAIN 12000] Mean Q Loss: 0.5741387124508619 
[TRAIN 13000] Mean Q Loss: 0.5807594513595105 
[TRAIN 14000] Mean Q Loss: 0.5774913482815027 
[TRAIN 15000] Mean Q Loss: 0.5696224720180034 
[TRAIN 16000] Mean Q Loss: 0.5836908711493015 
[TRAIN 17000] Mean Q Loss: 0.5828945854604244 
[TRAIN 18000] Mean Q Loss: 0.5717102480977774 
[TRAIN 19000] Mean Q Loss: 0.5711632730662822 
[TRAIN 20000] Mean Q Loss: 0.5519349473565817 
[TRAIN 21000] Mean Q Loss: 

In [18]:
env = ENVS_REPLAY_BUFFER[0][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[0][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=4, bias=True)
  )
)

In [19]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]


d:\Luiz\Faculdade\RL\OffRL\util\network.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states = torch.tensor(states, dtype=torch.long)  # Garante que é tensor de inteiros


[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: 0.06


In [20]:
extract_and_display(model, ENVS_REPLAY_BUFFER[0][0])

d:\Luiz\Faculdade\RL\OffRL\venv\lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at d:\Luiz\Faculdade\RL\OffRL\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## Taxi

In [28]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 300_000
GAMMA = 0.97  # Fator de desconto
TAU = 1e-2      # Taxa de atualização da target_network
ALPHA = 1.      # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [512, 256]
ISDISCRETE = True
# Extra:

In [29]:
ENVS_REPLAY_BUFFER[1][2].batch_size = BATCH_SIZE

In [30]:
config = get_config(ENVS_REPLAY_BUFFER[1][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [31]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[1][2])

[TRAIN 0] Mean Q Loss: 2.724316120147705 
[TRAIN 1000] Mean Q Loss: 1.9323909575939178 
[TRAIN 2000] Mean Q Loss: 1.882425920009613 
[TRAIN 3000] Mean Q Loss: 1.8884449652433395 
[TRAIN 4000] Mean Q Loss: 1.9026549785137177 
[TRAIN 5000] Mean Q Loss: 1.897267846107483 
[TRAIN 6000] Mean Q Loss: 1.8961939408779145 
[TRAIN 7000] Mean Q Loss: 1.909586605668068 
[TRAIN 8000] Mean Q Loss: 1.9190448307991028 
[TRAIN 9000] Mean Q Loss: 1.9189553217887878 
[TRAIN 10000] Mean Q Loss: 1.9285700632333755 
[TRAIN 11000] Mean Q Loss: 1.9331291455030442 
[TRAIN 12000] Mean Q Loss: 1.9333126780986787 
[TRAIN 13000] Mean Q Loss: 1.9402399237155914 
[TRAIN 14000] Mean Q Loss: 1.9351439144611358 
[TRAIN 15000] Mean Q Loss: 1.9384970465898514 
[TRAIN 16000] Mean Q Loss: 1.9286461151838303 
[TRAIN 17000] Mean Q Loss: 1.929705456018448 
[TRAIN 18000] Mean Q Loss: 1.9237619860172273 
[TRAIN 19000] Mean Q Loss: 1.9192200266122819 
[TRAIN 20000] Mean Q Loss: 1.9174528912305833 
[TRAIN 21000] Mean Q Loss: 1.92

In [32]:
env = ENVS_REPLAY_BUFFER[1][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[1][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=500, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=6, bias=True)
  )
)

In [33]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -10000.0


In [34]:
extract_and_display(model, ENVS_REPLAY_BUFFER[1][0])

## Cliffwalking

In [21]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 128
STEPS = 200_000
GAMMA = 0.98  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 1      # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 64]  # Tamanho da camada oculta
ISDISCRETE = True
# Extra:

In [22]:
ENVS_REPLAY_BUFFER[2][2].batch_size = BATCH_SIZE

In [23]:
config = get_config(ENVS_REPLAY_BUFFER[2][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [24]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[2][2])

[TRAIN 0] Mean Q Loss: 1.926201343536377 
[TRAIN 1000] Mean Q Loss: 1.983913958787918 
[TRAIN 2000] Mean Q Loss: 1.927588945865631 
[TRAIN 3000] Mean Q Loss: 1.9713338413238526 
[TRAIN 4000] Mean Q Loss: 1.9950470522642136 
[TRAIN 5000] Mean Q Loss: 2.019177780747414 
[TRAIN 6000] Mean Q Loss: 2.086139222860336 
[TRAIN 7000] Mean Q Loss: 2.0743441016674042 
[TRAIN 8000] Mean Q Loss: 2.139035302042961 
[TRAIN 9000] Mean Q Loss: 2.0675957074165345 
[TRAIN 10000] Mean Q Loss: 2.11224379837513 
[TRAIN 11000] Mean Q Loss: 2.0607930505275727 
[TRAIN 12000] Mean Q Loss: 2.076631922364235 
[TRAIN 13000] Mean Q Loss: 2.0689552100896837 
[TRAIN 14000] Mean Q Loss: 2.0586232172250747 
[TRAIN 15000] Mean Q Loss: 2.069195437669754 
[TRAIN 16000] Mean Q Loss: 2.028257346391678 
[TRAIN 17000] Mean Q Loss: 2.0492197116613386 
[TRAIN 18000] Mean Q Loss: 2.0514553836584093 
[TRAIN 19000] Mean Q Loss: 2.0205623923540115 
[TRAIN 20000] Mean Q Loss: 2.008441601276398 
[TRAIN 21000] Mean Q Loss: 2.008290989

In [25]:
env = ENVS_REPLAY_BUFFER[2][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[2][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=48, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=4, bias=True)
  )
)

In [26]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -10000.0


In [27]:
extract_and_display(model, ENVS_REPLAY_BUFFER[2][0])

## CartPole

In [35]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 50_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 0.5     # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 1e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 128]
ISDISCRETE = False
# Extra:

In [36]:
ENVS_REPLAY_BUFFER[3][2].batch_size = BATCH_SIZE

In [37]:
config = get_config(ENVS_REPLAY_BUFFER[3][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [38]:
ENVS_REPLAY_BUFFER[3][2].memory

deque([Experience(state=[0.0444495715200901, -0.027678364887833595, 0.00578987505286932, 0.042084816843271255], action=np.int64(0), reward=1.0, done=False, next_state=[0.04389600455760956, -0.22288286685943604, 0.0066315713338553905, 0.33658885955810547]),
       Experience(state=[0.04389600455760956, -0.22288286685943604, 0.0066315713338553905, 0.33658885955810547], action=np.int64(0), reward=1.0, done=False, next_state=[0.03943834826350212, -0.4180985689163208, 0.013363348320126534, 0.6313556432723999]),
       Experience(state=[0.03943834826350212, -0.4180985689163208, 0.013363348320126534, 0.6313556432723999], action=np.int64(0), reward=1.0, done=False, next_state=[0.031076375395059586, -0.613404393196106, 0.02599046193063259, 0.9282169342041016]),
       Experience(state=[0.031076375395059586, -0.613404393196106, 0.02599046193063259, 0.9282169342041016], action=np.int64(1), reward=1.0, done=False, next_state=[0.018808288499712944, -0.4186427891254425, 0.04455479979515076, 0.643813

In [39]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[3][2])

[TRAIN 0] Mean Q Loss: 1.0043326616287231 
[TRAIN 1000] Mean Q Loss: 0.7005414591431618 
[TRAIN 2000] Mean Q Loss: 0.7186437159180641 
[TRAIN 3000] Mean Q Loss: 0.7579090577363968 
[TRAIN 4000] Mean Q Loss: 0.7937349818944931 
[TRAIN 5000] Mean Q Loss: 0.8249645273685455 
[TRAIN 6000] Mean Q Loss: 0.8548590614199638 
[TRAIN 7000] Mean Q Loss: 0.8780542189478874 
[TRAIN 8000] Mean Q Loss: 0.913584353685379 
[TRAIN 9000] Mean Q Loss: 0.9245749883651734 
[TRAIN 10000] Mean Q Loss: 0.9417433308959007 
[TRAIN 11000] Mean Q Loss: 0.9648851878643036 
[TRAIN 12000] Mean Q Loss: 0.9778347088098526 
[TRAIN 13000] Mean Q Loss: 0.9847701951265335 
[TRAIN 14000] Mean Q Loss: 0.9989571861624718 
[TRAIN 15000] Mean Q Loss: 1.0026937589645386 
[TRAIN 16000] Mean Q Loss: 1.0023076053857802 
[TRAIN 17000] Mean Q Loss: 1.0097685179710387 
[TRAIN 18000] Mean Q Loss: 1.0119281520843506 
[TRAIN 19000] Mean Q Loss: 1.0305719563364983 
[TRAIN 20000] Mean Q Loss: 1.0367978775501252 
[TRAIN 21000] Mean Q Loss: 

In [40]:
env = ENVS_REPLAY_BUFFER[3][1]
model = MLP(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[3][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [41]:
evaluate_Continuous(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: 261.4


In [42]:
#extract_and_display(model, ENVS_REPLAY_BUFFER[3][0])
render_environment(model, ENVS_REPLAY_BUFFER[3][0])

Episódio 1: Recompensa = 137.0, Passos = 137
Episódio 2: Recompensa = 229.0, Passos = 229
Episódio 3: Recompensa = 157.0, Passos = 157
Episódio 4: Recompensa = 206.0, Passos = 206
Episódio 5: Recompensa = 422.0, Passos = 422


## LunarLander

In [43]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 50_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 0.5     # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 1e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 256]
ISDISCRETE = False
# Extra:

In [44]:
ENVS_REPLAY_BUFFER[4][2].batch_size = BATCH_SIZE

In [45]:
config = get_config(ENVS_REPLAY_BUFFER[4][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [46]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[4][2])

[TRAIN 0] Mean Q Loss: 1.980950117111206 
[TRAIN 1000] Mean Q Loss: 2.0390443966388703 
[TRAIN 2000] Mean Q Loss: 1.9443374800682067 
[TRAIN 3000] Mean Q Loss: 1.9036108098030091 
[TRAIN 4000] Mean Q Loss: 1.910914454460144 
[TRAIN 5000] Mean Q Loss: 1.9098856176137924 
[TRAIN 6000] Mean Q Loss: 1.9388884640932083 
[TRAIN 7000] Mean Q Loss: 1.9618278753757477 
[TRAIN 8000] Mean Q Loss: 2.001547687292099 
[TRAIN 9000] Mean Q Loss: 2.0590162163972856 
[TRAIN 10000] Mean Q Loss: 2.1151485838890074 
[TRAIN 11000] Mean Q Loss: 2.173246920108795 
[TRAIN 12000] Mean Q Loss: 2.2573153626918794 
[TRAIN 13000] Mean Q Loss: 2.3276251064538958 
[TRAIN 14000] Mean Q Loss: 2.4232866376638413 
[TRAIN 15000] Mean Q Loss: 2.5189243820905687 
[TRAIN 16000] Mean Q Loss: 2.5967798680067062 
[TRAIN 17000] Mean Q Loss: 2.7050629407167435 
[TRAIN 18000] Mean Q Loss: 2.827757777929306 
[TRAIN 19000] Mean Q Loss: 2.976853619337082 
[TRAIN 20000] Mean Q Loss: 3.112467025756836 
[TRAIN 21000] Mean Q Loss: 3.2897

In [47]:
env = ENVS_REPLAY_BUFFER[4][1]
model = MLP(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[4][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=4, bias=True)
  )
)

In [48]:
evaluate_Continuous(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -373.002293452655


In [49]:
#extract_and_display(model, ENVS_REPLAY_BUFFER[3][0])
render_environment(model, ENVS_REPLAY_BUFFER[4][0])

Episódio 1: Recompensa = -264.21819512582067, Passos = 381
Episódio 2: Recompensa = -272.6908653316161, Passos = 384
Episódio 3: Recompensa = -299.6239958973275, Passos = 639
Episódio 4: Recompensa = -313.73482090495065, Passos = 573
Episódio 5: Recompensa = -242.08849796862623, Passos = 417
